In [3]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
#client.list_experiments()
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1746540502605, experiment_id='1', last_update_time=1746540502605, lifecycle_stage='active', name='nyc-taxi-prediction-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1746524992992, experiment_id='0', last_update_time=1746524992992, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
client.create_experiment(name= "new_run_experiment")

'2'

In [7]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids = "1",
    filter_string = "",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metric.rmse asc"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 24457e356fcd49b08e363323da6db705, rmse: 6.3315
run id: 9ce852b9e5fd42d08552436bf409efaa, rmse: 6.3339
run id: 07e354c0902d4f51b78cdb2b6f15672d, rmse: 6.3598
run id: 70efad5458734cdb9f36adefcba11ced, rmse: 6.3640
run id: c7cbf98e58264c53985973153d50ad7a, rmse: 6.3665


In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
run_id = "c7cbf98e58264c53985973153d50ad7a"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri = model_uri, name = "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747044034224, current_stage='None', description=None, last_updated_timestamp=1747044034224, name='nyc-taxi-regressor', run_id='c7cbf98e58264c53985973153d50ad7a', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/c7cbf98e58264c53985973153d50ad7a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [15]:
# Moving the model stage through code here
client.get_registered_model("nyc-taxi-regressor")

<RegisteredModel: aliases={}, creation_timestamp=1747043145796, description='', last_updated_timestamp=1747044034224, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1747044034224, current_stage='None', description=None, last_updated_timestamp=1747044034224, name='nyc-taxi-regressor', run_id='c7cbf98e58264c53985973153d50ad7a', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/c7cbf98e58264c53985973153d50ad7a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>

In [17]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


/tmp/ipykernel_4200/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [22]:
client.transition_model_version_stage(
    name = model_name,
    version = 3,
    stage = "Staging",
    archive_existing_versions=False
)

/tmp/ipykernel_4200/739290423.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747044034224, current_stage='Staging', description=None, last_updated_timestamp=1747044586438, name='nyc-taxi-regressor', run_id='c7cbf98e58264c53985973153d50ad7a', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/c7cbf98e58264c53985973153d50ad7a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>